# 정령 시연 시나리오 (던전)  
## 1층 던전 입장  
- "인벤토리에 아이템이 뭐 있어?" (STT 사용 모습 보여줌)
- "xxxx 사용해" (장비무기 사용 상호작용)
- "던전의 밝기좀 켜줘" (던전 밝기 조절 on)

## 던전 플레이 도중
- "던전의 밝기좀 꺼줘" (던전 밝기 조절 off)

## 보스몬스터 조우
- "보스몬스터의 전투 패턴을 알려줘" 

In [1]:
import os, sys
from pathlib import Path

def find_src_folder():
    current = Path(os.getcwd()).resolve()
    for p in [current] + list(current.parents):
        src = p / "src"
        if src.exists():
            return src
    raise RuntimeError("src 폴더를 찾을 수 없습니다.")

src_path = find_src_folder()
sys.path.append(str(src_path))
from agents.fairy.dungeon.fairy_dungeon_agent import (
    graph_builder as dungeon_graph_builder,
)

from agents.fairy.util import add_human_message
from core.game_dto.z_muck_factory import MockFactory
from core.game_dto.StatData import StatData
from agents.fairy.fairy_state import DungeonPlayerState

from agents.fairy.interaction.fairy_interaction_agent import (
    graph_builder as interaction_builder,
)
from langchain.messages import HumanMessage
from agents.fairy.memory_messages import get_fairy_messages_dungeon
from core.common import get_inventory_items, get_skills


dungeon_graph = dungeon_graph_builder.compile()
interaction_graph = interaction_builder.compile()


async def invoke_graphs(
    question: str,
    target_monster_ids=[], 
    weapon=MockFactory.create_weapon(),
    currrRoom_id = 1,
):
    inventory_ids = [20, 40, 46, 67, 110]
    stats = StatData(
        strength=10,
        dexterity=10,
        intelligence=10
    )
    inventory = get_inventory_items(inventory_ids, stats)    
    skill_ids = [0, 1]
    skills = get_skills(skill_ids)

    player = DungeonPlayerState(
        playerId="BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
        heroineId=1,
        currRoomId=currrRoom_id,
        difficulty=0,
        stats=stats,
        skills=skills,
        weapon=weapon,
        inventory=inventory,
    )

    memories = get_fairy_messages_dungeon(
        player_id=player.playerId, heroine_id=player.heroineId, limit=4
    )

    inputs = {
        "messages": memories + [add_human_message(content=question)],
        "dungenon_player": player,
        "target_monster_ids": target_monster_ids,
    }

    dungeon_talk = await dungeon_graph.ainvoke(inputs)

    interaction_talk = interaction_graph.invoke(
        {
            "inventory": inventory_ids,
            "messages": memories + [HumanMessage(question)],
            "weapon": player.weapon,
            "stats": player.stats,
        }
    )
    print("기억::", memories)
    print("질문::", dungeon_talk["messages"][-2])
    print("응답::", dungeon_talk["messages"][-1])
    print(
        "상호 작용:",
        {
            "useItemId": interaction_talk["useItemId"],
            "roomLight": interaction_talk["roomLight"],
            "temp_use_item_id": interaction_talk["temp_use_item_id"],
            "is_item_use": interaction_talk["is_item_use"],
            "latency_create_temp_use_item_id": interaction_talk["latency_create_temp_use_item_id"]
        },
    )


/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:383: UserWarning: Valid config keys have changed in V2:
* 'orm_mode' has been renamed to 'from_attributes'
  warnings.warn(message, UserWarning)
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/main.py:250: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 357469.09it/s]
You're using a XLMRobertaTokenizerFast

In [2]:
#1
await invoke_graphs("인벤토리에 쌘무기 뭐 있어?")

총 질문 <Used Abilities>
['INTERACTION_HANDLER']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
  

In [3]:
#2
#FALSE
await invoke_graphs("그것 중에 가장 약한걸로 껴줘")

총 질문 <Used Abilities>
['INTERACTION_HANDLER']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
  

In [4]:
#3
from core.game_dto.z_muck_factory import MockFactory
await invoke_graphs('무기 교체 해줘!', weapon= MockFactory.create_yakk_weapon())

총 질문 <Used Abilities>
['INTERACTION_HANDLER']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 20,
    "weaponType": 3,
    "weaponName": "레어 드래곤슬레이어",
    "rarity": 

In [5]:
#4
await invoke_graphs('보스몬스터의 전투 패턴을 알려줘', target_monster_ids=[1001])

총 질문 <Used Abilities>
['MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "att

In [6]:
#5
await invoke_graphs('던전의 밝기좀 켜줘')

총 질문 <Used Abilities>
['INTERACTION_HANDLER']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
  

In [7]:
#6
await invoke_graphs("인벤토리에 뭐 있어?")

총 질문 <Used Abilities>
['INTERACTION_HANDLER']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
  

In [8]:
#7
await invoke_graphs("중량의 반지 사용해")

총 질문 <Used Abilities>
['INTERACTION_HANDLER']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
  

In [9]:
#8
await invoke_graphs("고양이의 털 사용해")

총 질문 <Used Abilities>
['INTERACTION_HANDLER']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
  

In [10]:
#9
await invoke_graphs("아이템 사용해")

총 질문 <Used Abilities>
['INTERACTION_HANDLER']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
  

In [11]:
#10
await invoke_graphs("숏쇼드랑 드래곤 슬레이어 데미지 몇인데")

총 질문 <Used Abilities>
['USAGE_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attac

In [12]:
#11
# FALSE (인터렉션은 다른걸 사용)
await invoke_graphs("그럼 둘중 약한걸로 써줘", weapon= MockFactory.create_power_weapon())

총 질문 <Used Abilities>
['INTERACTION_HANDLER', 'SMALLTALK']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 67,
    "weaponType": 4,
    "weaponName": "레전드 바이킹의 망치",


In [13]:
#12
await invoke_graphs("고급 드워프의 망치 사용해")

총 질문 <Used Abilities>
['INTERACTION_HANDLER']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
  

In [14]:
#13
await invoke_graphs("레어 쌍검 사용해", weapon= MockFactory.create_power_weapon())

총 질문 <Used Abilities>
['INTERACTION_HANDLER']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 67,
    "weaponType": 4,
    "weaponName": "레전드 바이킹의 망치",
    "rarity":

In [15]:
#14
await invoke_graphs("인벤토리에 아이템 뭐 있어?")

총 질문 <Used Abilities>
['INTERACTION_HANDLER']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
  

In [16]:
#15
await invoke_graphs("불 꺼줘")

총 질문 <Used Abilities>
['INTERACTION_HANDLER']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
  

In [17]:
#16
await invoke_graphs("스킬 사용법이 뭐야?")

총 질문 <Used Abilities>
['USAGE_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attac

In [18]:
#17
await invoke_graphs("저 보스몬스터는 뭐야?", target_monster_ids=[1000])

총 질문 <Used Abilities>
['MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "att

In [19]:
#18
await invoke_graphs("무기 스왑 해줘", target_monster_ids=[1000])

총 질문 <Used Abilities>
['INTERACTION_HANDLER', 'MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
  

In [20]:
#19
await invoke_graphs("인벤토리에 악세사리 뭐있어?", target_monster_ids=[1000])

총 질문 <Used Abilities>
['INTERACTION_HANDLER', 'MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
  

In [21]:
#20
await invoke_graphs("현재 방에 몬스터 얼마나 있어?", currrRoom_id=1)

총 질문 <Used Abilities>
['DUNGEON_NAVIGATOR']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    

In [22]:
#21
#FALSE
await invoke_graphs("저거 뭐야?", target_monster_ids=[1000])

총 질문 <Used Abilities>
['SMALLTALK', 'MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity"

In [23]:
#22
await invoke_graphs("사용키좀 알려줘", target_monster_ids=[1000])

총 질문 <Used Abilities>
['USAGE_GUIDE', 'MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarit

In [24]:
#23
await invoke_graphs("이벤트 어떻게 꺠?", target_monster_ids=[1000])

총 질문 <Used Abilities>
['EVENT_GUIDE', 'MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarit

In [25]:
#24
await invoke_graphs("내 인벤토리에 쓸만한 장비 2개만 말해봐")

총 질문 <Used Abilities>
['INTERACTION_HANDLER']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
  

In [26]:
#25
await invoke_graphs("둘중 쌘거 껴줘")

총 질문 <Used Abilities>
['INTERACTION_HANDLER', 'SMALLTALK']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "r

In [27]:
#26
await invoke_graphs("안녕 반가워!")

총 질문 <Used Abilities>
['SMALLTALK']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackP

In [28]:
#27
await invoke_graphs("방금 내가 뭐라고 했어?")

총 질문 <Used Abilities>
['SMALLTALK']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackP

In [29]:
#28
await invoke_graphs("스킬 사용해")

총 질문 <Used Abilities>
['INTERACTION_HANDLER']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
  

In [30]:
#29
await invoke_graphs("일반 대검과 일반 쌍검의 데미지가 어떻게 돼?")

총 질문 <Used Abilities>
['USAGE_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attac

In [31]:
#30
await invoke_graphs("배고프네 먹을것 추천해줘")

총 질문 <Used Abilities>
['SMALLTALK']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackP

In [32]:
#31
await invoke_graphs("내가 방금 질문한 내용이 뭐였지?")

총 질문 <Used Abilities>
['SMALLTALK']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackP

In [33]:
#32
await invoke_graphs("저 몬스터 뭐야?")

총 질문 <Used Abilities>
['MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "att

In [34]:
#33
await invoke_graphs("저기 있는 적 이름이 뭐야",target_monster_ids=[5])

총 질문 <Used Abilities>
['MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "att

In [35]:
#34
await invoke_graphs("스켈레톤 왜 이렇게 단단해",target_monster_ids=[0])

총 질문 <Used Abilities>
['MONSTER_GUIDE', 'SMALLTALK']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity"

In [36]:
#35
await invoke_graphs("저 몬스터 체력 얼마나 돼",target_monster_ids=[1])

총 질문 <Used Abilities>
['MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "att

In [37]:

#36
await invoke_graphs("공격력이 센 편이야?",target_monster_ids=[4])

총 질문 <Used Abilities>
['SMALLTALK', 'MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity"

In [38]:
#37
await invoke_graphs("움직임이 느린 이유가 있어?")

총 질문 <Used Abilities>
['SMALLTALK']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackP

In [39]:
#38
await invoke_graphs("저거 보스야?",target_monster_ids=[3])

총 질문 <Used Abilities>
['MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "att

In [40]:
#39
await invoke_graphs("보스면 패턴이 뭐야?",target_monster_ids=[1000])

총 질문 <Used Abilities>
['MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "att

In [41]:
#40
await invoke_graphs("약한 공격 방식이 있어?",target_monster_ids=[2])


총 질문 <Used Abilities>
['MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "att

In [42]:
#41
await invoke_graphs("저 몬스터는 근접이 위험해?", target_monster_ids=[1])

총 질문 <Used Abilities>
['MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "att

In [43]:
#42
await invoke_graphs("저거 공격 범위 넓어 보여?", target_monster_ids=[1001])

총 질문 <Used Abilities>
['MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "att

In [44]:
#43
await invoke_graphs("저 몬스터 스태거 잘 걸려?", target_monster_ids=[3])

총 질문 <Used Abilities>
['MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "att

In [45]:
#44
await invoke_graphs("경험치 많이 줘?",target_monster_ids=[1000])

총 질문 <Used Abilities>
['USAGE_GUIDE', 'MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarit

In [46]:
#45
await invoke_graphs("언데드 같은데 맞아?", target_monster_ids=[0])

총 질문 <Used Abilities>
['MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "att

In [47]:
#46
await invoke_graphs("저 적이 다른 몬스터보다 센 이유가 뭐야?", target_monster_ids=[1001])

총 질문 <Used Abilities>
['MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "att

In [48]:
#47
await invoke_graphs("여러 마리 같이 나오면 위험해?", target_monster_ids=[5])

총 질문 <Used Abilities>
['MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "att

In [49]:
#48
await invoke_graphs("지금 내 상태로 잡을 수 있어?", target_monster_ids=[4])

총 질문 <Used Abilities>
['INTERACTION_HANDLER', 'MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
  

In [50]:
#49
await invoke_graphs("공격 속도가 높은 편이야?", target_monster_ids=[4])

총 질문 <Used Abilities>
['SMALLTALK', 'MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity"

In [51]:
#50
await invoke_graphs("저거랑 전에 본 몬스터랑 뭐가 달라?", target_monster_ids=[3])

총 질문 <Used Abilities>
['MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "att

In [52]:
#51
await invoke_graphs("전투는 어떻게 하는 거야?")

총 질문 <Used Abilities>
['USAGE_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attac

In [53]:
#52
await invoke_graphs("스킬은 언제 쓰는 게 좋아")

총 질문 <Used Abilities>
['USAGE_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attac

In [54]:
#53
await invoke_graphs("이 게임에서 던전은 왜 도는 거야")

총 질문 <Used Abilities>
['USAGE_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attac

In [55]:
#54
await invoke_graphs("여기서 나가려면 어떻게 해야 돼")

총 질문 <Used Abilities>
['USAGE_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attac

In [56]:
#55
await invoke_graphs("아이템은 어떻게 사용하는 거야")

총 질문 <Used Abilities>
['USAGE_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attac

In [57]:
#56
await invoke_graphs("무기 바꾸면 뭐가 달라져")

총 질문 <Used Abilities>
['USAGE_GUIDE', 'INTERACTION_HANDLER']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    

In [58]:
#57
await invoke_graphs("공격 속도는 전투에 어떤 영향을 줘")

총 질문 <Used Abilities>
['USAGE_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attac

In [59]:
#58
await invoke_graphs("스태거가 정확히 뭐야")

총 질문 <Used Abilities>
['MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "att

In [60]:
#59
await invoke_graphs("지금 같은 상황에서 조심해야 할 게 있어")

총 질문 <Used Abilities>
['SMALLTALK']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackP

In [61]:
#60
await invoke_graphs("이 게임에서 중요한 스탯이 뭐야")

총 질문 <Used Abilities>
['USAGE_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attac

In [62]:
#61
await invoke_graphs("던전 클리어하면 뭐가 좋아")

총 질문 <Used Abilities>
['USAGE_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attac

In [63]:
#62
await invoke_graphs("보스 전투 전에 준비해야 할 게 있어")

총 질문 <Used Abilities>
['USAGE_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attac

In [64]:
#63
await invoke_graphs("이 상황에서 안전하게 싸우는 방법이 뭐야")

총 질문 <Used Abilities>
['USAGE_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attac

In [65]:
#64
#FALSE
await invoke_graphs("장비 등급 차이가 큰 편이야")

총 질문 <Used Abilities>
['SMALLTALK']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackP

In [66]:
#65
await invoke_graphs("난이도가 높아지면 뭐가 달라져")

총 질문 <Used Abilities>
['USAGE_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attac

In [67]:
#66
await invoke_graphs("이 던전 구조는 어떤 식이야")

총 질문 <Used Abilities>
['DUNGEON_NAVIGATOR']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    

In [68]:
#67
await invoke_graphs("전투 중에 위치 잡는 게 중요해")

총 질문 <Used Abilities>
['SMALLTALK']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackP

In [69]:
#68
await invoke_graphs("공격만 계속 해도 되는 거야")

총 질문 <Used Abilities>
['SMALLTALK']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackP

In [70]:
#69
await invoke_graphs("이 상태로 계속 진행해도 괜찮아")


총 질문 <Used Abilities>
['SMALLTALK']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackP

In [71]:
#70
await invoke_graphs("초반에 어떤 식으로 싸우는 게 좋아")


총 질문 <Used Abilities>
['USAGE_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attac

In [72]:
#71
await invoke_graphs("저거 몬스터 뭐야 좀 위험한거야")

총 질문 <Used Abilities>
['MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "att

In [73]:
#72
await invoke_graphs("불 좀 켜고 저기 적 정보 알려줘")

총 질문 <Used Abilities>
['INTERACTION_HANDLER', 'MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
  

In [74]:
#73
await invoke_graphs("다음 방 어디야 위험해 보여")

총 질문 <Used Abilities>
['DUNGEON_NAVIGATOR']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    

In [75]:
#74
await invoke_graphs("이거 이벤트야 그냥 지나가도 돼",currrRoom_id=2)

총 질문 <Used Abilities>
['EVENT_GUIDE', 'SMALLTALK']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 2,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 

In [76]:
#75
await invoke_graphs("저 몬스터 약점 뭐야 그리고 지금 싸워도 돼",target_monster_ids=[5])


총 질문 <Used Abilities>
['MONSTER_GUIDE', 'SMALLTALK']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity"

In [77]:
#76
#FALSE
await invoke_graphs("이 장치 뭐하는거야 건드리면 안되나",currrRoom_id=2)


총 질문 <Used Abilities>
['EVENT_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 2,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attac

In [78]:
#77
await invoke_graphs("나 지금 상태로 보스 잡을수있어")


총 질문 <Used Abilities>
['INTERACTION_HANDLER', 'SMALLTALK']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "r

In [79]:
#78
#FALSE (인터렉션은 불꺼짐)
await invoke_graphs("불 꺼도 돼? 근데 저기 뭐 나올거같은데")

총 질문 <Used Abilities>
['INTERACTION_HANDLER', 'SMALLTALK']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "r

In [80]:
#79
#FALSE (인터렉션은 사용)
await invoke_graphs("아이템 쓰고 싶은데 지금 써도 괜찮아")

총 질문 <Used Abilities>
['INTERACTION_HANDLER', 'SMALLTALK']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "r

In [81]:
#80
await invoke_graphs("여기서 뭐부터 해야돼 저거 몬스터부터야", currrRoom_id=2)


총 질문 <Used Abilities>
['DUNGEON_NAVIGATOR', 'SMALLTALK']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 2,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rar

In [82]:
#81
await invoke_graphs("이 방 이벤트면 실패하면 아픈거야", currrRoom_id=2)


총 질문 <Used Abilities>
['EVENT_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 2,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attac

In [83]:
#82
await invoke_graphs("다음 길 가기전에 조심할거 있어")

총 질문 <Used Abilities>
['SMALLTALK']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackP

In [84]:
#83
await invoke_graphs("저 몬스터 공격 빠른편이야 위험해")

총 질문 <Used Abilities>
['MONSTER_GUIDE', 'SMALLTALK']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity"

In [85]:
#84
await invoke_graphs("이거 그냥 싸우는거 맞지 다른거 없어")

총 질문 <Used Abilities>
['SMALLTALK']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackP

In [86]:
#85
await invoke_graphs("무기 바꾸는게 나아 지금 몬스터 보니까")

총 질문 <Used Abilities>
['INTERACTION_HANDLER', 'MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
  

In [87]:
#86
#FALSE (인터랙션은 불을 끔)
await invoke_graphs("여기 불 안켜도 진행은 되지")

총 질문 <Used Abilities>
['DUNGEON_NAVIGATOR', 'USAGE_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "r

In [88]:
#87
await invoke_graphs("이 상황에서 싸우는게 맞는지 모르겠어")

총 질문 <Used Abilities>
['SMALLTALK']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackP

In [89]:
#88
await invoke_graphs("저 장치 먼저 만지면 전투 나와",currrRoom_id=2)

총 질문 <Used Abilities>
['EVENT_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 2,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attac

In [90]:
#89
await invoke_graphs("지금 던전 계속 가도 되는거야")

총 질문 <Used Abilities>
['DUNGEON_NAVIGATOR']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    

In [91]:
#90
await invoke_graphs("저거 보스면 패턴 보고 들어가는게 나아")

총 질문 <Used Abilities>
['MONSTER_GUIDE', 'DUNGEON_NAVIGATOR']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    

In [92]:
#91
await invoke_graphs("나 좀 약해보이지 않아?")

총 질문 <Used Abilities>
['SMALLTALK']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackP

In [93]:
#92
await invoke_graphs("지금 상황 좀 빡센데")

총 질문 <Used Abilities>
['SMALLTALK']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackP

In [94]:
#93
await invoke_graphs("나 잘하고 있는거 맞지")

총 질문 <Used Abilities>
['SMALLTALK']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackP

In [95]:
#94
await invoke_graphs("이거 생각보다 무서운데")

BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<function=FairyDungeonIntentOutput>["SMALLTALK"]</function>'}}

In [ ]:
#95
await invoke_graphs("분위기 왜 이래")

In [ ]:
#96
await invoke_graphs("좀 긴장된다")

In [ ]:
#97
await invoke_graphs("여기 느낌 안좋아")

In [ ]:
#98
await invoke_graphs("열받네")

In [ ]:
#99
await invoke_graphs("나 실수한거 같아")

In [ ]:
#100
await invoke_graphs("지금 괜찮은 선택이었어")